In [1]:
import numpy as np
import json
import os
import copy
import pickle

import mesh_sampling
import trimesh
from shape_data import ShapeData

from autoencoder_dataset import autoencoder_dataset, cached_autoencoder_dataset
from torch.utils.data import DataLoader

from spiral_utils import get_adj_trigs, generate_spirals
from models import SpiralAutoencoderVariationalLoss
from train_funcs import train_variational_autoencoder_dataloader
from test_funcs import test_variational_autoencoder_dataloader


import torch
from tensorboardX import SummaryWriter

from sklearn.metrics.pairwise import euclidean_distances
meshpackage = 'mpi-mesh' # 'mpi-mesh', 'trimesh'
root_dir = '/home/jingwang/Data/data/'

dataset = 'FaceWarehouse'
name = 'real_variational'

torch.backends.cudnn.benchmark = True


GPU = True
os.environ["CUDA_VISIBLE_DEVICES"]="2"
device_idx = 0
print(torch.cuda.get_device_name(device_idx))

GeForce GTX TITAN X


In [2]:
args = {}

generative_model = 'variational_autoencoder'
downsample_method = 'COMA_downsample' # choose'COMA_downsample' or 'meshlab_downsample'


# below are the arguments for the DFAUST run
reference_mesh_file = os.path.join(root_dir, dataset, 'template', 'template.obj')
downsample_directory = os.path.join(root_dir, dataset,'template', downsample_method)
ds_factors = [4, 4, 4, 4]
step_sizes = [2, 2, 1, 1, 1]
filter_sizes_enc = [[3, 16, 32, 64, 128],[[],[],[],[],[]]]
filter_sizes_dec = [[128, 64, 32, 32, 16],[[],[],[],[],3]]
dilation_flag = True
if dilation_flag:
    dilation=[2, 2, 1, 1, 1] 
else:
    dilation = None
reference_points = [[5930]]# [[3567,4051,4597]] # [[414]]  # used for COMA with 3 disconnected components

args = {'generative_model': generative_model,
        'name': name, 'data': os.path.join(root_dir, dataset, 'preprocessed',name),
        'results_folder':  os.path.join(root_dir, dataset,'results/spirals_'+ generative_model),
        'reference_mesh_file':reference_mesh_file, 'downsample_directory': downsample_directory,
        'checkpoint_file': 'checkpoint',
        'seed':2, 'loss':'l1',
        'batch_size': 16, 'num_epochs':300, 'eval_frequency':200, 'num_workers': 40,
        'filter_sizes_enc': filter_sizes_enc, 'filter_sizes_dec': filter_sizes_dec,
        'nz': 128, # 100 identity + 46 expression 
        'ds_factors': ds_factors, 'step_sizes' : step_sizes, 'dilation': dilation,
        
        'lr':1e-3, 
        'regularization': 5e-5,         
        'scheduler': True, 'decay_rate': 0.99,'decay_steps':1,  
        'resume': True,
        
        'mode':'test', 'shuffle': False, 'nVal': 100, 'normalization': True,
        'write_mesh': True,
        'lambda_var': 1e-5,
        'worst_face_num': 20,
        'use_cache': True}

args['results_folder'] = os.path.join(args['results_folder'],'latent_'+str(args['nz']))
    
if not os.path.exists(os.path.join(args['results_folder'])):
    os.makedirs(os.path.join(args['results_folder']))

summary_path = os.path.join(args['results_folder'],'summaries',args['name'])
if not os.path.exists(summary_path):
    os.makedirs(summary_path)  
    
checkpoint_path = os.path.join(args['results_folder'],'checkpoints', args['name'])
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
    
samples_path = os.path.join(args['results_folder'],'samples', args['name'])
if not os.path.exists(samples_path):
    os.makedirs(samples_path)
    
prediction_path = os.path.join(args['results_folder'],'predictions', args['name'])
if not os.path.exists(prediction_path):
    os.makedirs(prediction_path)

if not os.path.exists(downsample_directory):
    os.makedirs(downsample_directory)

downsample_mesh_path = os.path.join(args['results_folder'],'downsample_mesh', args['name'])
if not os.path.exists(downsample_mesh_path):
    os.makedirs(downsample_mesh_path)
    
worst_mesh_path = os.path.join(args['results_folder'],'worst_test', args['name'])
if not os.path.exists(downsample_mesh_path):
    os.makedirs(downsample_mesh_path)

In [3]:
np.random.seed(args['seed'])
print("Loading data .. ")
if not os.path.exists(args['data']+'/mean.npy') or not os.path.exists(args['data']+'/std.npy'):
    shapedata =  ShapeData(nVal=args['nVal'], 
                          train_file=args['data']+'/train.npy', 
                          test_file=args['data']+'/test.npy', 
                          reference_mesh_file=args['reference_mesh_file'],
                          normalization = args['normalization'],
                          meshpackage = meshpackage, load_flag = True)
    np.save(args['data']+'/mean.npy', shapedata.mean)
    np.save(args['data']+'/std.npy', shapedata.std)
else:
    shapedata = ShapeData(nVal=args['nVal'], 
                         train_file=args['data']+'/train.npy',
                         test_file=args['data']+'/test.npy', 
                         reference_mesh_file=args['reference_mesh_file'],
                         normalization = args['normalization'],
                         meshpackage = meshpackage, load_flag = False)
    shapedata.mean = np.load(args['data']+'/mean.npy')
    shapedata.std = np.load(args['data']+'/std.npy')
    shapedata.n_vertex = shapedata.mean.shape[0]
    shapedata.n_features = shapedata.mean.shape[1]

if not os.path.exists(os.path.join(args['downsample_directory'],'downsampling_matrices.pkl')):
    if shapedata.meshpackage == 'trimesh':
        raise NotImplementedError('Rerun with mpi-mesh as meshpackage')
    print("Generating Transform Matrices ..")
    if downsample_method == 'COMA_downsample':
        M,A,D,U,F = mesh_sampling.generate_transform_matrices(shapedata.reference_mesh, args['ds_factors'])
        if args['write_mesh']:
            import openmesh
            for i in range(len(M)):
                mesh = openmesh.TriMesh(points=M[i].v,face_vertex_indices=M[i].f)
                openmesh.write_mesh(os.path.join(args['results_folder'],'downsample_mesh',args['name'], '%d.obj'%i),mesh)
    with open(os.path.join(args['downsample_directory'],'downsampling_matrices.pkl'), 'wb') as fp:
        M_verts_faces = [(M[i].v, M[i].f) for i in range(len(M))]
        pickle.dump({'M_verts_faces':M_verts_faces,'A':A,'D':D,'U':U,'F':F}, fp)
else:
    print("Loading Transform Matrices ..")
    with open(os.path.join(args['downsample_directory'],'downsampling_matrices.pkl'), 'rb') as fp:
        #downsampling_matrices = pickle.load(fp,encoding = 'latin1')
        downsampling_matrices = pickle.load(fp)
            
    M_verts_faces = downsampling_matrices['M_verts_faces']
    if shapedata.meshpackage == 'mpi-mesh':
        from psbody.mesh import Mesh
        M = [Mesh(v=M_verts_faces[i][0], f=M_verts_faces[i][1]) for i in range(len(M_verts_faces))]
    elif shapedata.meshpackage == 'trimesh':
        M = [trimesh.base.Trimesh(vertices=M_verts_faces[i][0], faces=M_verts_faces[i][1], process = False) for i in range(len(M_verts_faces))]
    A = downsampling_matrices['A']
    D = downsampling_matrices['D']
    U = downsampling_matrices['U']
    F = downsampling_matrices['F']
        
# Needs also an extra check to enforce points to belong to different disconnected component at each hierarchy level
print("Calculating reference points for downsampled versions..")
for i in range(len(args['ds_factors'])):
    if shapedata.meshpackage == 'mpi-mesh':
        dist = euclidean_distances(M[i+1].v, M[0].v[reference_points[0]])
    elif shapedata.meshpackage == 'trimesh':
        dist = euclidean_distances(M[i+1].vertices, M[0].vertices[reference_points[0]])
    reference_points.append(np.argmin(dist,axis=0).tolist())



Loading data .. 
Loading Transform Matrices ..
Calculating reference points for downsampled versions..


In [4]:
if shapedata.meshpackage == 'mpi-mesh':
    sizes = [x.v.shape[0] for x in M]
elif shapedata.meshpackage == 'trimesh':
    sizes = [x.vertices.shape[0] for x in M]
Adj, Trigs = get_adj_trigs(A, F, shapedata.reference_mesh, meshpackage = shapedata.meshpackage)

spirals_np, spiral_sizes,spirals = generate_spirals(args['step_sizes'], 
                                                    M, Adj, Trigs, 
                                                    reference_points = reference_points, 
                                                    dilation = args['dilation'], random = False, 
                                                    meshpackage = shapedata.meshpackage, 
                                                    counter_clockwise = True)

bU = []
bD = []
for i in range(len(D)):
    d = np.zeros((1,D[i].shape[0]+1,D[i].shape[1]+1))
    u = np.zeros((1,U[i].shape[0]+1,U[i].shape[1]+1))
    d[0,:-1,:-1] = D[i].todense()
    u[0,:-1,:-1] = U[i].todense()
    d[0,-1,-1] = 1
    u[0,-1,-1] = 1
    bD.append(d)
    bU.append(u)


spiral generation for hierarchy 0 (11510 vertices) finished
spiral generation for hierarchy 1 (2878 vertices) finished
spiral generation for hierarchy 2 (720 vertices) finished
spiral generation for hierarchy 3 (180 vertices) finished
spiral generation for hierarchy 4 (45 vertices) finished
spiral sizes for hierarchy 0:  14
spiral sizes for hierarchy 1:  13
spiral sizes for hierarchy 2:  9
spiral sizes for hierarchy 3:  9
spiral sizes for hierarchy 4:  9


In [5]:
torch.manual_seed(args['seed'])

if GPU:
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    device = torch.device("cuda:"+str(device_idx) if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
print(device)

tspirals = [torch.from_numpy(s).long().to(device) for s in spirals_np]
tD = [torch.from_numpy(s).float().to(device) for s in bD]
tU = [torch.from_numpy(s).float().to(device) for s in bU]

cuda:0


In [6]:
# Building model, optimizer, and loss function

if args['use_cache']:
    dataset_train = cached_autoencoder_dataset(root_dir = args['data'], points_dataset = 'train',
                                               shapedata = shapedata,
                                               normalization = args['normalization'], device=device)
    dataloader_train = DataLoader(dataset_train, batch_size=args['batch_size'],\
                                         shuffle = args['shuffle'], num_workers=0)
else:
    dataset_train = autoencoder_dataset(root_dir = args['data'], points_dataset = 'train',
                                               shapedata = shapedata,
                                               normalization = args['normalization'])
    dataloader_train = DataLoader(dataset_train, batch_size=args['batch_size'],\
                                         shuffle = args['shuffle'], num_workers = args['num_workers'],pin_memory=True)

if args['use_cache']:
    dataset_val = cached_autoencoder_dataset(root_dir = args['data'], points_dataset = 'val', 
                                             shapedata = shapedata,
                                             normalization = args['normalization'], device=device)
    dataloader_val = DataLoader(dataset_val, batch_size=args['batch_size'],\
                                         shuffle = False, num_workers=0)
else:
    dataset_val = autoencoder_dataset(root_dir = args['data'], points_dataset = 'val', 
                                             shapedata = shapedata,
                                             normalization = args['normalization'])
    dataloader_val = DataLoader(dataset_val, batch_size=args['batch_size'],\
                                         shuffle = False, num_workers = args['num_workers'],pin_memory=True)


dataset_test = autoencoder_dataset(root_dir = args['data'], points_dataset = 'test',
                                          shapedata = shapedata,
                                          normalization = args['normalization'])

dataloader_test = DataLoader(dataset_test, batch_size=args['batch_size'],\
                                     shuffle = False, num_workers = args['num_workers'],  pin_memory=True)



if 'autoencoder' == args['generative_model']:
        model = SpiralAutoencoder(filters_enc = args['filter_sizes_enc'],   
                                  filters_dec = args['filter_sizes_dec'],
                                  latent_size=args['nz'],
                                  sizes=sizes,
                                  spiral_sizes=spiral_sizes,
                                  spirals=tspirals,
                                  D=tD, U=tU).to(device)
elif 'variational_autoencoder' == args['generative_model']:
    model = SpiralAutoencoderVariationalLoss(filters_enc = args['filter_sizes_enc'],   
                                  filters_dec = args['filter_sizes_dec'],
                                  latent_size=args['nz'],
                                  sizes=sizes,
                                  spiral_sizes=spiral_sizes,
                                  spirals=tspirals,
                                  D=tD, U=tU).to(device)
 
    
optim = torch.optim.Adam(model.parameters(),lr=args['lr'],weight_decay=args['regularization'])
if args['scheduler']:
    scheduler=torch.optim.lr_scheduler.StepLR(optim, args['decay_steps'],gamma=args['decay_rate'])
else:
    scheduler = None

if args['loss']=='l1':
    def loss_l1(outputs, targets):
        L = torch.abs(outputs - targets).mean()
        return L 
    loss_fn = loss_l1

def var_loss_fn(mu, logvar):
    var_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return var_loss



In [7]:
params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of parameters is: {}".format(params)) 
print(model)
# print(M[4].v.shape)

Total number of parameters is: 2480323
SpiralAutoencoderVariationalLoss(
  (conv): ModuleList(
    (0): SpiralConv(
      (conv): Linear(in_features=42, out_features=16, bias=True)
      (activation): ELU(alpha=1.0)
    )
    (1): SpiralConv(
      (conv): Linear(in_features=208, out_features=32, bias=True)
      (activation): ELU(alpha=1.0)
    )
    (2): SpiralConv(
      (conv): Linear(in_features=288, out_features=64, bias=True)
      (activation): ELU(alpha=1.0)
    )
    (3): SpiralConv(
      (conv): Linear(in_features=576, out_features=128, bias=True)
      (activation): ELU(alpha=1.0)
    )
  )
  (fc_latent_enc_mu): Linear(in_features=5888, out_features=128, bias=True)
  (fc_latent_enc_logvar): Linear(in_features=5888, out_features=128, bias=True)
  (fc_latent_dec): Linear(in_features=128, out_features=5888, bias=True)
  (dconv): ModuleList(
    (0): SpiralConv(
      (conv): Linear(in_features=1152, out_features=64, bias=True)
      (activation): ELU(alpha=1.0)
    )
    (1):

In [8]:
if args['mode'] == 'train':
    writer = SummaryWriter(summary_path)
    with open(os.path.join(args['results_folder'],'checkpoints', args['name'] +'_params.json'),'w') as fp:
        saveparams = copy.deepcopy(args)
        json.dump(saveparams, fp)
        
    if args['resume']:
            print('loading checkpoint from file %s'%(os.path.join(checkpoint_path,args['checkpoint_file'])))
            checkpoint_dict = torch.load(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar'),map_location=device)
            start_epoch = checkpoint_dict['epoch'] + 1
            model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])
            optim.load_state_dict(checkpoint_dict['optimizer_state_dict'])
            scheduler.load_state_dict(checkpoint_dict['scheduler_state_dict'])
            print('Resuming from epoch %s'%(str(start_epoch)))     
    else:
        start_epoch = 0
        
    if args['generative_model'] == 'autoencoder':
        train_autoencoder_dataloader(dataloader_train, dataloader_val,
                          device, model, optim, loss_fn,
                          bsize = args['batch_size'],
                          start_epoch = start_epoch,
                          n_epochs = args['num_epochs'],
                          eval_freq = args['eval_frequency'],
                          scheduler = scheduler,
                          writer = writer,
                          save_recons=True,
                          shapedata=shapedata,
                          metadata_dir=checkpoint_path, samples_dir=samples_path,
                          checkpoint_path = args['checkpoint_file'])
    elif args['generative_model'] == 'variational_autoencoder':
        train_variational_autoencoder_dataloader(dataloader_train, dataloader_val,
                          device, model, optim, loss_fn, var_loss_fn, args['lambda_var'],
                          bsize = args['batch_size'],
                          start_epoch = start_epoch,
                          n_epochs = args['num_epochs'],
                          eval_freq = args['eval_frequency'],
                          scheduler = scheduler,
                          writer = writer,
                          save_recons=True,
                          shapedata=shapedata,
                          metadata_dir=checkpoint_path, samples_dir=samples_path,
                          checkpoint_path = args['checkpoint_file'])

loading checkpoint from file /home/jingwang/Data/data/FaceWarehouse/results/spirals_variational_autoencoder/latent_128/checkpoints/real_variational/checkpoint


  0%|          | 0/580 [00:00<?, ?it/s]

Resuming from epoch 120


100%|██████████| 7/7 [00:00<00:00,  7.65it/s]


epoch 120 | tr 0.0935972279644 | val 0.12669129163


...

epoch 397 | tr 0.0680566154099 | val 0.126565287411


100%|██████████| 7/7 [00:00<00:00,  8.10it/s]


epoch 398 | tr 0.0680607265504 | val 0.126024469435


100%|██████████| 7/7 [00:00<00:00,  8.10it/s]


epoch 399 | tr 0.06799378485 | val 0.126273058951
~FIN~


lambda_var=1e-5 | epoch 399 | tr 0.06799378485 | val 0.126273058951

In [ ]:
model.module.fc_latent_dec.weight.device

In [9]:
if args['mode'] == 'test': # lambda_var = 1e-5
    print('loading checkpoint from file %s'%(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar')))
    checkpoint_dict = torch.load(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar'),map_location=device)
    model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])
        
    predictions, norm_l1_loss, l2_loss = test_variational_autoencoder_dataloader(device, model, dataloader_test, 
                                                                     shapedata, worst_path=worst_mesh_path, 
                                                                     worst_face_num=args['worst_face_num'],
                                                                     mm_constant = 100)    
    np.save(os.path.join(prediction_path,'predictions'), predictions)   
        
    print('autoencoder: normalized loss', norm_l1_loss)
    
    print('autoencoder: euclidean distance in mm=', l2_loss)

loading checkpoint from file /home/jingwang/Data/data/FaceWarehouse/results/spirals_variational_autoencoder/latent_128/checkpoints/real_variational/checkpoint.pth.tar


100%|██████████| 42/42 [00:08<00:00,  7.63it/s]


('loss for ', 402, tensor(3.7335, device='cuda:0'))
('loss for ', 169, tensor(3.7734, device='cuda:0'))
('loss for ', 364, tensor(3.7402, device='cuda:0'))
('loss for ', 537, tensor(3.7736, device='cuda:0'))
('loss for ', 370, tensor(3.8049, device='cuda:0'))
('loss for ', 612, tensor(3.7892, device='cuda:0'))
('loss for ', 194, tensor(3.8605, device='cuda:0'))
('loss for ', 499, tensor(3.8726, device='cuda:0'))
('loss for ', 8, tensor(3.8141, device='cuda:0'))
('loss for ', 628, tensor(3.8478, device='cuda:0'))
('loss for ', 389, tensor(3.9959, device='cuda:0'))
('loss for ', 50, tensor(3.8525, device='cuda:0'))
('loss for ', 560, tensor(3.8142, device='cuda:0'))
('loss for ', 583, tensor(3.8838, device='cuda:0'))
('loss for ', 125, tensor(3.8742, device='cuda:0'))
('loss for ', 341, tensor(3.8894, device='cuda:0'))
('loss for ', 602, tensor(3.9182, device='cuda:0'))
('loss for ', 626, tensor(3.9905, device='cuda:0'))
('loss for ', 559, tensor(3.8887, device='cuda:0'))
('loss for ', 3

In [8]:
if args['mode'] == 'test':
    print('loading checkpoint from file %s'%(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar')))
    checkpoint_dict = torch.load(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar'),map_location=device)
    model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])
        
    predictions, norm_l1_loss, l2_loss = test_variational_autoencoder_dataloader(device, model, dataloader_test, 
                                                                     shapedata, worst_path=worst_mesh_path, 
                                                                     worst_face_num=args['worst_face_num'],
                                                                     mm_constant = 100)    
    np.save(os.path.join(prediction_path,'predictions'), predictions)   
        
    print('autoencoder: normalized loss', norm_l1_loss)
    
    print('autoencoder: euclidean distance in mm=', l2_loss)

loading checkpoint from file /home/jingwang/Data/data/FaceWarehouse/results/spirals_variational_autoencoder/latent_128/checkpoints/real_variational/checkpoint.pth.tar


100%|██████████| 42/42 [00:08<00:00,  7.62it/s]


('loss for ', 560, tensor(3.3856, device='cuda:0'))
('loss for ', 169, tensor(3.3871, device='cuda:0'))
('loss for ', 499, tensor(3.3962, device='cuda:0'))
('loss for ', 402, tensor(3.4166, device='cuda:0'))
('loss for ', 537, tensor(3.3890, device='cuda:0'))
('loss for ', 364, tensor(3.4537, device='cuda:0'))
('loss for ', 612, tensor(3.4090, device='cuda:0'))
('loss for ', 370, tensor(3.4655, device='cuda:0'))
('loss for ', 390, tensor(3.4357, device='cuda:0'))
('loss for ', 389, tensor(3.5490, device='cuda:0'))
('loss for ', 628, tensor(3.4595, device='cuda:0'))
('loss for ', 602, tensor(3.4861, device='cuda:0'))
('loss for ', 583, tensor(3.4888, device='cuda:0'))
('loss for ', 50, tensor(3.4964, device='cuda:0'))
('loss for ', 626, tensor(3.4939, device='cuda:0'))
('loss for ', 341, tensor(3.5481, device='cuda:0'))
('loss for ', 125, tensor(3.5830, device='cuda:0'))
('loss for ', 8, tensor(3.4934, device='cuda:0'))
('loss for ', 194, tensor(3.5734, device='cuda:0'))
('loss for ', 5

In [10]:
if args['mode'] == 'test': # test with train set
    print('loading checkpoint from file %s'%(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar')))
    checkpoint_dict = torch.load(os.path.join(checkpoint_path,args['checkpoint_file']+'.pth.tar'),map_location=device)
    model.load_state_dict(checkpoint_dict['autoencoder_state_dict'])
        
    predictions, norm_l1_loss, l2_loss = test_variational_autoencoder_dataloader(device, model, dataloader_train, 
                                                                     shapedata, worst_path=worst_mesh_path, 
                                                                     worst_face_num=args['worst_face_num'],
                                                                     mm_constant = 100)    
    np.save(os.path.join(prediction_path,'predictions'), predictions)   
        
    print('autoencoder: normalized loss', norm_l1_loss)
    
    print('autoencoder: euclidean distance in mm=', l2_loss)

loading checkpoint from file /home/jingwang/Data/data/FaceWarehouse/results/spirals_variational_autoencoder/latent_128/checkpoints/real_variational/checkpoint.pth.tar


100%|██████████| 580/580 [01:30<00:00,  6.25it/s]


('loss for ', 7306, tensor(0.6339, device='cuda:0'))
('loss for ', 1443, tensor(0.6429, device='cuda:0'))
('loss for ', 5314, tensor(0.6481, device='cuda:0'))
('loss for ', 5167, tensor(0.6456, device='cuda:0'))
('loss for ', 1052, tensor(0.6705, device='cuda:0'))
('loss for ', 1533, tensor(0.6584, device='cuda:0'))
('loss for ', 4689, tensor(0.6725, device='cuda:0'))
('loss for ', 5009, tensor(0.6617, device='cuda:0'))
('loss for ', 7357, tensor(0.6693, device='cuda:0'))
('loss for ', 7875, tensor(0.8278, device='cuda:0'))
('loss for ', 3114, tensor(0.7798, device='cuda:0'))
('loss for ', 3660, tensor(0.8747, device='cuda:0'))
('loss for ', 9272, tensor(0.6907, device='cuda:0'))
('loss for ', 6394, tensor(0.7031, device='cuda:0'))
('loss for ', 8612, tensor(0.6756, device='cuda:0'))
('loss for ', 2876, tensor(0.7130, device='cuda:0'))
('loss for ', 8032, tensor(0.6918, device='cuda:0'))
('loss for ', 737, tensor(0.6941, device='cuda:0'))
('loss for ', 4046, tensor(0.7354, device='cuda

In [12]:
dataset_train.paths[7357]

'5705'

In [37]:
from pprint import pprint
pprint(list(model.modules()))

[DataParallel(
  (module): SpiralAutoencoder(
    (conv_0): SpiralConv(
      (conv): Linear(in_features=42, out_features=16, bias=True)
      (activation): ELU(alpha=1.0)
    )
    (conv_1): SpiralConv(
      (conv): Linear(in_features=208, out_features=32, bias=True)
      (activation): ELU(alpha=1.0)
    )
    (conv_2): SpiralConv(
      (conv): Linear(in_features=288, out_features=64, bias=True)
      (activation): ELU(alpha=1.0)
    )
    (conv_3): SpiralConv(
      (conv): Linear(in_features=576, out_features=128, bias=True)
      (activation): ELU(alpha=1.0)
    )
    (fc_latent_enc): Linear(in_features=5888, out_features=146, bias=True)
    (fc_latent_dec): Linear(in_features=146, out_features=5888, bias=True)
    (dconv): ModuleList(
      (0): SpiralConv(
        (conv): Linear(in_features=1152, out_features=64, bias=True)
        (activation): ELU(alpha=1.0)
      )
      (1): SpiralConv(
        (conv): Linear(in_features=576, out_features=32, bias=True)
        (activation

In [10]:
# from optimize_to_get_result import optimize_to_get_result

import torch
import torch.optim as optim
import torch.nn as nn


def optimize_to_get_result(model, loss_fun, z_dim, device, targets, n_iter=10, output_loss=True):
    model.eval()
    
#     return torch.empty(0), model(targets)
    
    model.only_encode(True)
    z = model(targets)
    model.only_encode(False)
    
    model.only_decode(True)
    z_param = nn.Parameter(z)
    optimizer = optim.LBFGS(params=[z_param])
    for it in range(n_iter):
        def closure():
            optimizer.zero_grad()
            outputs = model(z_param)
            outputs = outputs.reshape(targets.shape) # decode, don't know shape
            loss = loss_fun(outputs, targets)
            loss.backward()
            if output_loss:
                print('loss ', loss.item())
            return loss
        optimizer.step(closure)
    outputs = model(z_param)
    outputs = outputs.reshape(targets.shape)
    
    model.only_decode(False)
    return z_param.data, outputs

def work_optimize():

    test_dataset = dataset_test
    test_ids =[560,169,499,402,537,364,612,370,390,389,628,602,583,50, 626,341,125,8,194,559]
    inputs = torch.cat([test_dataset[idx]['points'].to(device).unsqueeze(0) for idx in test_ids])
    inputs = inputs.to(device)

    print(inputs.shape) # fixme

    shapedata_mean = torch.Tensor(shapedata.mean).to(device)
    shapedata_std = torch.Tensor(shapedata.std).to(device)

    z, outputs = optimize_to_get_result(model, loss_fn, args['nz'], device,
                          inputs)
    from pprint import pprint
    pprint(z)

    outputs = outputs[:,:-1]
    inputs = inputs[:,:-1]

    old_outputs = outputs
    old_inputs = inputs

    outputs = outputs * shapedata_std + shapedata_mean
    inputs = inputs * shapedata_std + shapedata_mean

    per_l2_loss = torch.sqrt(torch.sum((outputs - inputs)**2, dim=2))

    for i in range(len(test_ids)):
        print(test_ids[i], torch.mean(per_l2_loss[i]))

    shapedata.save_meshes(os.path.join(worst_mesh_path, 'opt_input'),old_inputs.detach().cpu().numpy(),test_ids)
    shapedata.save_meshes(os.path.join(worst_mesh_path, 'opt_output'),old_outputs.detach().cpu().numpy(),test_ids)

work_optimize()

torch.Size([20, 11511, 3])
('loss ', 0.4986802637577057)
('loss ', 0.49858540296554565)
('loss ', 0.5245462656021118)
('loss ', 0.42691150307655334)
('loss ', 0.43026289343833923)
('loss ', 0.4063626825809479)
('loss ', 0.40158355236053467)
('loss ', 0.39085909724235535)
('loss ', 0.3910890221595764)
('loss ', 0.38281163573265076)
('loss ', 0.38037484884262085)
('loss ', 0.3779156506061554)
('loss ', 0.375139057636261)
('loss ', 0.37313753366470337)
('loss ', 0.36904338002204895)
('loss ', 0.3667895197868347)
('loss ', 0.36510035395622253)
('loss ', 0.3637033998966217)
('loss ', 0.3623831570148468)
('loss ', 0.36083289980888367)
('loss ', 0.35960763692855835)
('loss ', 0.35859280824661255)
('loss ', 0.357584148645401)
('loss ', 0.35685405135154724)
('loss ', 0.35606375336647034)
('loss ', 0.3552369475364685)
('loss ', 0.35456404089927673)
('loss ', 0.35376542806625366)
('loss ', 0.3530873954296112)
('loss ', 0.35253989696502686)
('loss ', 0.35190171003341675)
('loss ', 0.35117784142494

In [10]:
# from optimize_to_get_result import optimize_to_get_result
# lambda_var = 1e-5

import torch
import torch.optim as optim
import torch.nn as nn


def optimize_to_get_result(model, loss_fun, z_dim, device, targets, n_iter=10, output_loss=True):
    model.eval()
    
#     return torch.empty(0), model(targets)
    
    model.only_encode(True)
    z = model(targets)
    model.only_encode(False)
    
    model.only_decode(True)
    z_param = nn.Parameter(z)
    optimizer = optim.LBFGS(params=[z_param])
    for it in range(n_iter):
        def closure():
            optimizer.zero_grad()
            outputs = model(z_param)
            outputs = outputs.reshape(targets.shape) # decode, don't know shape
            loss = loss_fun(outputs, targets)
            loss.backward()
            if output_loss:
                print('loss ', loss.item())
            return loss
        optimizer.step(closure)
    outputs = model(z_param)
    outputs = outputs.reshape(targets.shape)
    
    model.only_decode(False)
    return z_param.data, outputs

def work_optimize():

    test_dataset = dataset_test
    test_ids =[560,169,499,402,537,364,612,370,390,389,628,602,583,50, 626,341,125,8,194,559]
    inputs = torch.cat([test_dataset[idx]['points'].to(device).unsqueeze(0) for idx in test_ids])
    inputs = inputs.to(device)

    print(inputs.shape) # fixme

    shapedata_mean = torch.Tensor(shapedata.mean).to(device)
    shapedata_std = torch.Tensor(shapedata.std).to(device)

    z, outputs = optimize_to_get_result(model, loss_fn, args['nz'], device,
                          inputs)
    from pprint import pprint
    pprint(z)

    outputs = outputs[:,:-1]
    inputs = inputs[:,:-1]

    old_outputs = outputs
    old_inputs = inputs

    outputs = outputs * shapedata_std + shapedata_mean
    inputs = inputs * shapedata_std + shapedata_mean

    per_l2_loss = torch.sqrt(torch.sum((outputs - inputs)**2, dim=2))

    for i in range(len(test_ids)):
        print(test_ids[i], torch.mean(per_l2_loss[i]))

    shapedata.save_meshes(os.path.join(worst_mesh_path, 'opt_input'),old_inputs.detach().cpu().numpy(),test_ids)
    shapedata.save_meshes(os.path.join(worst_mesh_path, 'opt_output'),old_outputs.detach().cpu().numpy(),test_ids)

work_optimize()

torch.Size([20, 11511, 3])
('loss ', 0.5567759275436401)
('loss ', 0.5563845634460449)
('loss ', 0.5783909559249878)
('loss ', 0.496934175491333)
('loss ', 0.48533686995506287)
('loss ', 0.47421765327453613)
('loss ', 0.47161945700645447)
('loss ', 0.46805039048194885)
('loss ', 0.4635757803916931)
('loss ', 0.4615626633167267)
('loss ', 0.4599860608577728)
('loss ', 0.4563556909561157)
('loss ', 0.45438069105148315)
('loss ', 0.4522996246814728)
('loss ', 0.4508543014526367)
('loss ', 0.44892674684524536)
('loss ', 0.4474303722381592)
('loss ', 0.4460495412349701)
('loss ', 0.44529640674591064)
('loss ', 0.44481122493743896)
('loss ', 0.44385385513305664)
('loss ', 0.4430656433105469)
('loss ', 0.44228968024253845)
('loss ', 0.4414037764072418)
('loss ', 0.44081467390060425)
('loss ', 0.4402202367782593)
('loss ', 0.4397233724594116)
('loss ', 0.4393344819545746)
('loss ', 0.43894192576408386)
('loss ', 0.4384746253490448)
('loss ', 0.4380544126033783)
('loss ', 0.43765899538993835)
(